# Google Gemini SDK Introduction

This notebook demonstrates how to use the Google Gemini AI SDK to interact with the Gemini model in both text and audio modes.

## Setup
First, we'll install the required package and initialize the client with our API key.

In [2]:
!pip install -U -q google-genai

In [3]:
from google import genai
# client = genai.Client(vertexai=True, location='us-central1', http_options= {'api_version': 'v1beta'})

## Text Interaction Example

Below we'll demonstrate how to have a text conversation with Gemini. The code:
1. Sets up a configuration for text responses
2. Opens an async connection to the model
3. Sends a message and receives the response in chunks
4. Prints each chunk of the response as it arrives

In [10]:
PROJECT_ID = "<YOUR_GOOGLE_CLOUD_PROJECT_ID>"
LOCATION = "us-central1"

In [9]:
model_id = "gemini-2.0-flash-exp"

client = genai.Client(vertexai=True, location=LOCATION, project=PROJECT_ID)
config = {"response_modalities": ["TEXT"]}

async with client.aio.live.connect(model=model_id, config=config) as session:
    message = "Hello? Gemini, are you there?"
    print("> ", message, "\n")
    await session.send(input=message, end_of_turn=True)

    async for response in session.receive():
        print(response.text)

>  Hello? Gemini, are you there? 

Yes
, I am here. What would you like to talk about?

None


## Audio Generation Example

Now we'll see how to generate audio responses from Gemini. This section:
1. Creates a wave file handler to save the audio
2. Configures the model for audio output
3. Sends a text prompt and receives audio data
4. Saves the audio chunks and plays them in the notebook

Note: Make sure your browser's audio is enabled to hear the responses.

In [11]:
import contextlib
import wave


@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

In [20]:
from IPython.display import display, Audio

config = {"response_modalities": ["AUDIO"]}


async with client.aio.live.connect(model=model_id, config=config) as session:
  file_name = 'audio.wav'
  with wave_file(file_name) as wav:
    message = "Hello? Gemini are you there?"
    print("> ", message, "\n")
    await session.send(input=message, end_of_turn=True)

    first = True
    async for response in session.receive():
      if response.data is not None:
        model_turn = response.server_content.model_turn
        if first:
          print(model_turn.parts[0].inline_data.mime_type)
          first = False
        print('.', end='.')
        wav.writeframes(response.data)


display(Audio(file_name, autoplay=True))

>  Hello? Gemini are you there? 

None
audio/pcm
..None
..None
..None
..None
..None
..None
..None
..None
..None
..None
..None
..None
..None


# Text and Audio Interaction Example

This section demonstrates how to receive both text and audio responses from Gemini. The code:
1. Sets up a configuration for text and audio responses
2. Opens an async connection to the model
3. Sends a message and receives the responses in chunks
4. Prints each text chunk of the response as it arrives
5. Collects the audio chunks and plays them in the notebook




In [22]:
from IPython.display import display, Audio

config = {"response_modalities": ["TEXT", "AUDIO"]}


async with client.aio.live.connect(model=model_id, config=config) as session:
  file_name = 'audio.wav'
  with wave_file(file_name) as wav:
    message = "Hello? Gemini are you there?"
    print("> ", message, "\n")
    await session.send(input=message, end_of_turn=True)

    first = True
    async for response in session.receive():
      if response.data is not None:
        model_turn = response.server_content.model_turn
        if first:
          print(model_turn.parts[0].inline_data.mime_type)
          first = False
        print('.', end='.')
        wav.writeframes(response.data)
      else:
        print(response.text)


display(Audio(file_name, autoplay=True))

>  Hello? Gemini are you there? 

Yes
, I'
m here
. What
 would
 you like to talk
audio/pcm
.. about?

............................None
